In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-crf

**get data**

In [3]:
data_link = "/content/drive/MyDrive/largeProject/code/NER/id_ner_new.csv"

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchcrf import CRF

In [5]:
df = pd.read_csv(data_link)

In [6]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,tokens,ner_tags
0,"['Chỉ_tiêu', 'PAKH/1000', 'thuê_bao/ngày', 'dị...","[6, 5, 3, 3, 3, 3, 6, 6, 2, 6, 6, 6, 6, 6, 6, ..."
1,"['Về', 'hiện_trạng', 'chỉ', '1', 'chỉ_tiêu', '...","[6, 6, 6, 6, 6, 5, 3, 3, 3, 3, 3, 3, 3, 3, 6, ..."
2,"['Tháng', '2', '2020', 'có_thể', 'cho', 'tui',...","[6, 10, 7, 6, 6, 6, 6, 6, 5, 3, 3, 3, 3, 3, 3,..."
3,"['Kết_quả', 'của', 'chỉ_tiêu', 'Tổng_đài', '18...","[6, 6, 6, 5, 3, 6, 6, 6, 6, 7, 6, 6, 6, 6, 10, 7]"
4,"['Xin', 'hỏi', 'tháng', '3', '2020', 'chỉ_tiêu...","[6, 6, 6, 10, 7, 6, 5, 3, 3, 3, 3, 3, 3, 3, 3,..."


In [7]:
df['tokens'] = df['tokens'].map(lambda x : eval(x))
df['ner_tags'] = df['ner_tags'].map(lambda x : eval(x))

In [8]:
len(df['tokens'][0]) == len(df['ner_tags'][0])

True

In [9]:
import pickle

with open('/content/drive/MyDrive/largeProject/code/NER/l2i_new.pkl','rb') as f:
  l2i = pickle.load(f)

with open('/content/drive/MyDrive/largeProject/code/NER/i2l_new.pkl','rb') as f:
  i2l = pickle.load(f)

In [10]:
l2i

{'I_<tên cụm chỉ tiêu>': 0,
 'I_<đơn vị>': 1,
 'B_<tên tổng công ty>': 2,
 'I_<tên chỉ tiêu>': 3,
 'B_<quý>': 4,
 'B_<tên chỉ tiêu>': 5,
 'O': 6,
 'B_<năm>': 7,
 'B_<đơn vị>': 8,
 'B_<năm m>': 9,
 'B_<tháng>': 10,
 'B_<tháng 1>': 11,
 'B_<tên cụm chỉ tiêu>': 12}

In [11]:
i2l

{0: 'I_<tên cụm chỉ tiêu>',
 1: 'I_<đơn vị>',
 2: 'B_<tên tổng công ty>',
 3: 'I_<tên chỉ tiêu>',
 4: 'B_<quý>',
 5: 'B_<tên chỉ tiêu>',
 6: 'O',
 7: 'B_<năm>',
 8: 'B_<đơn vị>',
 9: 'B_<năm m>',
 10: 'B_<tháng>',
 11: 'B_<tháng 1>',
 12: 'B_<tên cụm chỉ tiêu>'}

**Dataset**

In [12]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [13]:
import transformers

In [14]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 16
BASE_MODEL_PATH = "vinai/phobert-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
EPOCHS = 10

**Dataset**

In [16]:
import torch

class EntityDataset:
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        tags = self.tags[item]

        ids = []
        target_tag = []

        for i, s in enumerate(text):
            inputs = tokenizer.encode(s, add_special_tokens=False)
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [0] + ids + [2] # : 0, : 2, : 1
        target_tag = [12] + target_tag + [12] # O: 12

        mask = [1] * len(ids) # 1: not masked, 0: masked
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([1] * padding_len) # padding
        mask = mask + ([0] * padding_len) # 1: not masked, 0: masked
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([12] * padding_len) # O: 12

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

**Model**

In [17]:
import torch.nn.functional as F
log_soft = F.log_softmax

In [18]:
def loss_fn(output, target, mask, num_labels):
    lfn = torch.nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss


class EntityModel(torch.nn.Module):
    def __init__(self, num_tag):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.bert = transformers.AutoModel.from_pretrained(BASE_MODEL_PATH,return_dict=False)
        # unfreeze bert
        for param in self.bert.parameters():
            param.requires_grad = False
        self.bert_drop = torch.nn.Dropout(0.3)
        self.out_tag = torch.nn.Linear(768, self.num_tag)
        self.crf = CRF(self.num_tag, batch_first = True)

    def forward_custom(self, ids, mask, token_type_ids, target_tag):
        o, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo_tag = self.bert_drop(o)
        tag = self.out_tag(bo_tag)

        loss = -self.crf(log_soft(tag, 2), target_tag, mask=mask.type(torch.uint8), reduction='mean')
        prediction = self.crf.decode(tag, mask=mask.type(torch.uint8))
        return prediction, loss

    def forward(self, ids, mask, token_type_ids, target_tag):
        o, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo_tag = self.bert_drop(o)
        tag = self.out_tag(bo_tag)
        loss = loss_fn(tag, target_tag, mask, self.num_tag)

        return tag, loss

**Function training**

In [19]:
from tqdm import tqdm

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        _, loss = model.forward_custom(**data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)


def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        _, loss = model.forward_custom(**data)
        final_loss += loss.item()
    return final_loss / len(data_loader)

**Train,val,test**

In [20]:
import pandas as pd
import numpy as np

import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import KFold

In [21]:
sentences = df["tokens"].to_list()
tag = df["ner_tags"].to_list()

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sentences, tag, test_size=0.05, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

In [23]:
num_tag = len(i2l)

**New optimizer**

In [ ]:
LR_BERT = 3e-5
L2_BERT = 1e-5

LR_LINEAR = 7e-4
L2_LINEAR = 1e-3

LR_CRF = 7e-4
L2_CRF = 1e-3

cnt = -1
num_layer = 199 #num of layers
# for param in model.named_parameters():
#     cnt += 1
#     if cnt>=num_layer:
#         param[1].requires_grad = True
#     else:
#         param[1].requires_grad = True
#     print(cnt,param[0],'\t',param[1].requires_grad)

model = EntityModel(num_tag=num_tag)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

FINETUNING = True
if FINETUNING:
    param_optimizer1 = list(model.named_parameters())[:num_layer]
    param_optimizer2 = list(model.named_parameters())[num_layer:num_layer+2]
    param_optimizer3 = list(model.named_parameters())[num_layer+2:]
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        #BERT
        {'params': [p for n, p in param_optimizer1 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': L2_BERT},
        {'params': [p for n, p in param_optimizer1 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0},

        #LINEAR
        {'params': [p for n, p in param_optimizer2 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': L2_LINEAR,
         'lr': LR_LINEAR},
        {'params': [p for n, p in param_optimizer2 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':LR_LINEAR},

        #CRF
        {'params': [p for n, p in param_optimizer3 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': L2_CRF,
         'lr':LR_CRF},
        {'params': [p for n, p in param_optimizer3 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':LR_CRF}
    ]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=LR_BERT,
    eps=1e-8
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


**Not k-fold**

In [39]:
train_sentences = X_train
train_tag = y_train
test_sentences = X_val
test_tag = y_val

train_dataset = EntityDataset(texts=train_sentences, tags=train_tag)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

test_dataset = EntityDataset(texts=test_sentences, tags=test_tag)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EntityModel(num_tag=num_tag)
model.load_state_dict(torch.load("/content/phobert_fold1.pth"))
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
          {
              "params": [
                  p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.001,
          },
          {
              "params": [
                  p for n, p in param_optimizer if any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.0,
          },
      ]

num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

epoch_id = 1
best_loss = np.inf

MODEL_PATH = '/content/phobert_fold1.pth'

for epoch in range(EPOCHS):
    train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
    test_loss = eval_fn(test_data_loader, model, device)
    print(f"Epoch {epoch_id}: Train Loss = {train_loss} Valid Loss = {test_loss}")
    if test_loss < best_loss:
      torch.save(model.state_dict(), MODEL_PATH)
      best_loss = test_loss
    epoch_id += 1

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 357/357 [01:04<00:00,  5.56it/s]


Epoch 1: Train Loss = 8.715924742654169 Valid Loss = 7.085635166542203


100%|██████████| 357/357 [01:05<00:00,  5.46it/s]


Epoch 2: Train Loss = 7.859437252457017 Valid Loss = 6.473284644573009


100%|██████████| 357/357 [01:04<00:00,  5.50it/s]


...

Epoch 8: Train Loss = 6.066476886397766 Valid Loss = 5.060011052283921


100%|██████████| 357/357 [01:01<00:00,  5.77it/s]


Epoch 9: Train Loss = 5.99564790007495 Valid Loss = 5.007221323101461


  8%|▊         | 138/1693 [01:24<15:53,  1.63it/s]


KeyboardInterrupt: ignored

**Kfold**

In [ ]:
num_tag = len(i2l)
kf = KFold(n_splits=5)

for train, test in kf.split(sentences):
  train_sentences = []
  test_sentences = []
  train_tag = []
  test_tag = []

  for id in train:
    train_sentences.append(sentences[id])
    train_tag.append(tag[id])

  for id in test:
    test_sentences.append(sentences[id])
    test_tag.append(tag[id])

  train_dataset = EntityDataset(texts=train_sentences, tags=train_tag)
  train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

  test_dataset = EntityDataset(texts=test_sentences, tags=test_tag)
  test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model = EntityModel(num_tag=num_tag)

  model.to(device)

  param_optimizer = list(model.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]

  num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
  optimizer = AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

  epoch_id = 1
  best_loss = np.inf

  MODEL_PATH = '/content/phobert_fold1.pth'

  for epoch in range(32):
      train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
      test_loss = eval_fn(test_data_loader, model, device)
      print(f"Epoch {epoch_id}: Train Loss = {train_loss} Valid Loss = {test_loss}")
      if test_loss < best_loss:
        torch.save(model.state_dict(), MODEL_PATH)
        best_loss = test_loss
      epoch_id += 1

**Inference**

In [26]:
def infer(sentence,num_tag=34):
  segmented_sentence = sentence.split(' ')
  test_dataset = EntityDataset(texts=[segmented_sentence], tags=[[0] * len(segmented_sentence)])
  # model = EntityModel(num_tag=num_tag)
  # model.load_state_dict(torch.load(MODEL_PATH))
  # model.to(device)

  tags = []
  with torch.no_grad():
    data = test_dataset[0]
    for k, v in data.items():
      data[k] = v.to(device).unsqueeze(0)
    tag, _ = model.forward_custom(**data)

  decoded_sentence = []
  for tokenized_word in test_dataset[0]["ids"]:
    decoded_word = tokenizer.decode([tokenized_word])
    decoded_sentence.append(decoded_word)

  print("{:15} {:5}".format("Word", "Tag"))
  print("="*30)
  for w, t in zip(decoded_sentence, tags):
    if w == "": break
    print("{:15}:{:5}".format(w, t))

In [40]:
test_dataset = EntityDataset(texts=X_test, tags=y_test)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

In [41]:
model1 = EntityModel(num_tag=num_tag)
model1.load_state_dict(torch.load(MODEL_PATH))
model1.to(device)

EntityModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [42]:
pred_tag = []
with torch.no_grad():
  for data in tqdm(test_dataset):
    for k, v in data.items():
      data[k] = v.to(device).unsqueeze(0)
    tag, _ = model1.forward_custom(**data)
    pred_tag.append(tag)

100%|██████████| 6000/6000 [06:27<00:00, 15.48it/s]


In [43]:
predict_tag = []
for tag, sentence in zip(pred_tag, X_test):
  tags = []
  tag = tag[0]
  for ele in tag:
    tags.append(i2l[ele])
  predict_tag.append(tags[:len(sentence) + 2])

In [44]:
target_tag = []

for dataset, sentence in zip(test_dataset, X_test):
  tag = dataset['target_tag'][:len(sentence) + 2]
  tags = []
  for ele in tag:
    tags.append(i2l[int(ele)])
  target_tag.append(tags)

In [32]:
print(len(target_tag))
len(predict_tag)

6000


6000

In [ ]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 11.9 MB/s eta 0:00:00


In [45]:
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder

In [46]:
new_target = []
for tag_lst in target_tag:
    for tag in tag_lst:
        new_target.append(tag)

new_predict = []
for tag_lst in predict_tag:
    for tag in tag_lst:
        new_predict.append(tag)

In [35]:
len(new_predict)

163475

In [36]:
len(new_target)

163475

In [47]:
print(classification_report(new_target, new_predict, labels=list(l2i.keys())))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

I_<tên cụm chỉ tiêu>       0.97      0.40      0.56     11239
          I_<đơn vị>       0.00      0.00      0.00        20
B_<tên tổng công ty>       0.96      0.99      0.98      3529
    I_<tên chỉ tiêu>       0.81      0.99      0.89     31169
             B_<quý>       0.91      0.96      0.93       920
    B_<tên chỉ tiêu>       0.82      0.94      0.88      4914
                   O       0.99      0.99      0.99     92102
             B_<năm>       0.98      0.98      0.98      6055
          B_<đơn vị>       0.62      0.37      0.46       130
           B_<năm m>       0.98      0.96      0.97       257
           B_<tháng>       0.97      0.97      0.97      4730
         B_<tháng 1>       1.00      0.24      0.39        58
B_<tên cụm chỉ tiêu>       0.99      0.92      0.95      8352

            accuracy                           0.94    163475
           macro avg       0.85      0.75      0.77    163475
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Model**

In [ ]:
import shutil
shutil.copy('/content/phobert_fold1.pth', '/content/drive/MyDrive/largeProject/code/NER/phobert_fold_crf_latest.pth')

'/content/drive/MyDrive/largeProject/code/NER/phobert_fold_crf_latest.pth'

In [48]:
import shutil
shutil.copy('/content/phobert_fold1.pth', '/content/drive/MyDrive/largeProject/code/phobert_fold_crf_new1.pth')

'/content/drive/MyDrive/largeProject/code/phobert_fold_crf_new1.pth'

In [ ]:
!ls /content/drive/MyDrive/largeProject/code/NER/

data_processing.ipynb  id_ner.csv	      phobert_fold1.pth
dataset		       l2i.pkl		      phobert_fold2.pth
dataset1	       ner_crf.ipynb	      phobert_fold_crf1.pth
i2l.pkl		       ner.ipynb	      phobert_fold_crf2.pth
id_ner_2000.csv        phobert_fold1_crf.pth
